# Quiz 2

In [1]:
import turicreate

In [2]:
sales = turicreate.SFrame('data/home_data.sframe')

## Pregunta 1

1. **Selección y resumen de estadísticas:** en el Notebook que se utilizó en el módulo, se descubrió qué vecindario (código postal) de Seattle tenía el precio promedio de venta de vivienda más alto. Ahora, tome los datos de ventas, seleccione solo las casas con este código postal y calcule el precio promedio. Guarde este resultado para responder la prueba al final.

In [3]:
# Se agrupan los datos por su 'código zip'.
# Se calcula el numero de casas en cada codigo zip.
# Se calcula el precio promedio de casas en cada codigo zip.
user_count = sales.groupby('zipcode', 
                           {     'count':turicreate.aggregate.COUNT(),  
                            'mean_price':turicreate.aggregate.MEAN('price')}) 

MaxPrice = user_count['mean_price'].max()
user_count[user_count['mean_price']==MaxPrice].head()

zipcode,count,mean_price
98039,50,2160606.6


## Pregunta 2

2. **Filtrado de datos:** una de las características clave que usamos en nuestro modelo fue la cantidad de pies cuadrados de cada espacio ("sqft_living") en la casa. Para esta parte, se usa la idea de filtrar (seleccionar) datos.  En particular, se utilizarán filtros lógicos para seleccionar filas de un SFrame.
    + Utilizando dichos filtros, primero seleccione las casas que tienen "sqft_living" más de 2000 pies cuadrados pero no mayores de 4000 pies cuadrados.
    + ¿Qué fracción de todas las casas tiene "sqft_living" en este rango? 

In [4]:
sales_filtered = sales[(sales['sqft_living']>2000)&(sales['sqft_living']<4000)]
Nsales = sales.shape[0]
Nsales_filtered = sales_filtered.shape[0]
p = Nsales_filtered/Nsales

In [5]:
print("Numero de salas filtradas: ", Nsales_filtered)
print("          Numero de salas: ", Nsales)
print("               Proporción: ", round(p,4))

Numero de salas filtradas:  9111
          Numero de salas:  21613
               Proporción:  0.4216


## Pregunta 3

3. **Creación de un modelo de regresión con varias características más:** en el Notebook de muestra, se creó dos modelos de regresión para predecir los precios de la vivienda, uno usando solo "sqft_living" y el otro usando algunas características más, este conjunto fue llamado:

    `my_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode']`

    Ahora, volviendo al conjunto de datos original, creará un modelo con las siguientes características:

    `advanced_features = [...]`

    + Calcule el **RMSE (error cuadrático medio)** en test_data para el modelo que usa solo my_features, y para el que usa advanced_features.

    $$ RMSE = \sqrt{\frac{RSS}{N}} $$

    + ¿Cuál es la diferencia en RMSE entre el modelo entrenado con **my_features** y el entrenado con **advanced_features**? 

### Partición de los datos

In [6]:
# Se dividen aleatoriamente los datos en un conjunto de entrenamiento (80%) 
# y un conjunto de prueba (20%).
training_set, test_set = sales.random_split(.8,seed=0)

### Modelo 1

Se crea un modelo de regresión lineal para predecir el **precio** (objetivo) con las características: `my_features`.

In [7]:
# Caracteristicas
my_features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','zipcode']

In [8]:
# Modelo de regresion lineal
my_features_model = turicreate.linear_regression.create(
    training_set,
    target='price',
    features=my_features,
    validation_set = None,
    verbose=False)

In [9]:
# Calcula el error
e_model1 = my_features_model.evaluate(test_set)
RMSE_model1 = e_model1['rmse']
RMSE_model1

180439.07296639626

### Modelo 2

Se crea un modelo de regresión lineal para predecir el **precio** (objetivo) con las características: `advanced_features`.

In [10]:
# Caracteristicas avanzadas
advanced_features = [
    'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode',
    'condition', # condition of house
    'grade', # measure of quality of construction
    'waterfront', # waterfront property
    'view', # type of view
    'sqft_above', # square feet above ground
    'sqft_basement', # square feet in basement
    'yr_built', # the year built
    'yr_renovated', # the year renovated
    'lat', 'long', # the lat-long of the parcel
    'sqft_living15', # average sq.ft. of 15 nearest neighbors
    'sqft_lot15', # average lot size of 15 nearest neighbors 
]

In [11]:
# Modelo de regresion lineal
advanced_features_model = turicreate.linear_regression.create(
    training_set,
    target='price',
    features=advanced_features,
    validation_set = None,
    verbose=False)

In [12]:
# Calcula el error
e_model2 = advanced_features_model.evaluate(test_set)
RMSE_model2 = e_model2['rmse']
RMSE_model2

155269.65792732083

### Diferencia de errores

In [13]:
# Diferencia de RMSE del modelo 1 y modelo 2_
difference = RMSE_model1 - RMSE_model2
difference

25169.415039075422